# SHELL-AI- HACKATHON FOR SUSTAINABLE AND AFFORDABLE ENERGY
ELECTRIC VEHICLE CHARGING POINT CHALLENGE



In [64]:
import pandas as pd  # for data manipulation
import numpy as np    # for mathematical calculation
#import seaborn as sns   # for data visualisation
#import matplotlib.pyplot as plt   #for plotting graph
#%matplotlib inline 
#matplotlib inline ensures command in cells bellow the cell that outputs a plot does not affect the plot
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
#from pandas_profiling import ProfileReport
import warnings   # to ignore warnings

#import all libraries and modules at the beginning of the code

In [5]:
#Loading in the data sets
Demand_History = pd.read_csv("Demand_History.csv")
Existing = pd.read_csv("EEVI2018.csv")
sample_submission = pd.read_csv('sample_submission.csv')

In [6]:
Demand_History.head() #Previewing the dataset

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563


In [7]:
Demand_History.isnull().sum() #checking for missing values

demand_point_index    0
x_coordinate          0
y_coordinate          0
2010                  0
2011                  0
2012                  0
2013                  0
2014                  0
2015                  0
2016                  0
2017                  0
2018                  0
dtype: int64

In [8]:
Demand_History.describe(include='all') #descriptive statistics

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,4096.00000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000
mean,2047.50000,32.000000,32.000000,1.993306,4.017967,7.552549,19.323394,31.891712,42.249471,58.718724,75.805574,88.264071
std,1182.55768,18.475209,18.475209,1.849883,4.003144,7.658317,19.080365,31.270080,39.501338,56.133565,76.390728,79.513884
min,0.00000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1023.75000,16.250000,16.250000,0.474756,0.898346,1.613829,4.480010,6.958344,10.634926,15.245858,22.668290,25.275349
50%,2047.50000,32.000000,32.000000,1.163430,2.134510,3.854949,10.045907,17.872258,24.146146,31.755415,37.285351,54.567937
75%,3071.25000,47.750000,47.750000,3.665382,6.996849,13.579687,34.418367,56.016385,77.892036,108.528007,138.112588,158.637564
max,4095.00000,63.500000,63.500000,6.641083,14.961144,28.383260,70.789208,117.063595,148.491476,219.652855,302.729826,317.422418


In [9]:
def calculate_Distij(demand_point_x=Demand_History['x_coordinate'], 
                    demand_point_y=Demand_History['y_coordinate'],
                    supply_point_x = Existing['x_coordinate'],
                    supply_point_y=Existing['y_coordinate']): 
    # for x coord
    newdf = pd.DataFrame()
    lis = []
    for i in demand_point_x:
        for j in supply_point_x:
            lis.append(i-j)
    lisarr = np.asarray(lis)
    #print(lisarr)
    # for y coord
    lis1 = []
    for i in demand_point_y:
        for j in supply_point_y:
            lis1.append(i-j)
    lisarr1 = np.asarray(lis1)
    lis2 = []
    #Calculating linear distances between two points
    for x, y in zip(lisarr, lisarr1):
        lis2.append(np.sqrt(x**2 + y**2))
    Distij = np.asarray(lis2)
    #dist1 is the copy of the reshaped dist value
    Distij = Distij.reshape(4096, 100)
    #print(dist1.shape)
    return Distij

In [10]:
x = calculate_Distij()

In [11]:
def truncate_min_k(x, k, inplace=False):
    m, n = x.shape
    # get (unsorted) indices of top-k values
    mink_indices = np.argpartition(x, k, axis=1)[:, :k]
    # get k-th value
    rows, _ = np.indices((m, k))
    kth_vals = x[rows, mink_indices].max(axis=1)
    # get boolean mask of values smaller than k-th
    is_greater_than_kth = x > kth_vals[:, None]
    # replace mask by 0
    if not inplace:
        return np.where(is_greater_than_kth, 0, x)
    x[is_greater_than_kth] = 0
    return x

In [12]:
k=10
min_k = truncate_min_k(x, k=k)
min_k.shape

(4096, 100)

In [13]:
def summation_Dist():
    real_sum = []
    for i in range(4096):
        real_sum.append(min_k[i].sum())
        
    sum_dist = np.asarray(real_sum)[:, None]
    return sum_dist

In [14]:
sum_dist = summation_Dist()

In [15]:
def supply_demand(Dforecast=Demand_History['2018'].values[:, None]):  
    """
    Constraint5: 
    inputs:
    trunmin: This is the truncated minimum distances of each demand points from
    every supply points
    di: This is the forecasted demands for a particular year
    sum_dist: This is the sum of the truncated distances of each demand point from every 
    supply points reshaped into 4096 X 1 to match the broadcastable shape of 'trunmin'
    
    Return:
        demand satisfied by each supply point by the demand point
    """
    sum_dist = summation_Dist()
    min_k = truncate_min_k(x, k).reshape(4096, -1)    
    lis = []
    for j in range(4096):
        for distance in min_k[j]: #changed to minfive
            #for x, y, z in zip(min_sup, sum_dist[0], di[0]):
              #  lis.append((x*z)/y)
            lis.append((distance*Dforecast[j])/sum_dist[j])
 
    
    return np.asarray(lis)

In [16]:
DSij = supply_demand()
DSij.shape

(409600, 1)

In [17]:
Existing['scs_cap'] = Existing['existing_num_SCS']*200
Existing['fcs_cap'] = Existing['existing_num_FCS']*400
Existing['total_cap'] = Existing['scs_cap']+ Existing['fcs_cap']
Existing['parking_cap'] = (Existing['total_parking_slots'] - Existing['existing_num_SCS']) * 400 + (Existing['existing_num_SCS']) * 200
Existing['available_cap'] = Existing['parking_cap'] - Existing['total_cap']

In [18]:
Demand_History['2018'].shape

(4096,)

In [19]:
def add_infrastructure(Dforecast=Demand_History['2018'].values[:, None]):
    """
    purpose: This function calculates the number of additional infrastructure 
    needed at every(100) supply points over all the demand points.
    Particularly optimizing for more FCS if more infrastructure is
    needed
    inputs:
    sup_cap:
    This is the overall calculated supply capacity returned by our model(calculation parameters)
    scs_cap{constant}:
    This is the supply or charging capacity of slow charging station at every supply point
    const_total{constant}:
    This is the constant total charging capacity at every supply point
    """
    newval =  supply_demand(Dforecast)
    newval = newval.reshape(100, -1)
    DS = np.sum(newval, 1)
    smax= Existing['total_cap'].values
    lis = []# for fast charging stations
    for DSij, Smax in zip(DS, smax):
    
        result = Smax - DSij
        if result >= 0:
            lis.append(0)
        else:
            result = abs(result)
            lis.append((result//400) + 1) #+1 removed

        
    
    more_infra = np.asarray(lis) # capacity to add
    return more_infra

In [20]:
to_add = add_infrastructure()
to_add

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  2.,  3.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  1.,  0.,  1.,  5.,  0.,  0.,  1.,  2.,  5.,  1.,  1.,  7.,
        2.,  1., 11.,  0.,  1., 11.,  1.,  6., 11.,  0.,  1.,  1.,  9.,
        9.,  0.,  0., 15.,  2., 11., 10.,  4., 11., 12.,  0.,  9., 10.,
        0., 17.,  0.,  2.,  8.,  2.,  4., 10.,  5., 17.,  0.,  9.,  6.,
        0.,  3.,  7.,  0.,  2.,  4.,  1.,  0.,  0.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [22]:
def excess_charging_cap(Dforecast=Demand_History['2018'].values[:, None], k=k):
    #Existing['new_FCS'] = Existing['existing_num_FCS'].values + to_add
    to_add = add_infrastructure(Dforecast)
    Existing['add_cap'] = to_add * 400
    Existing['excess_cap'] = Existing['available_cap'] - Existing['add_cap']
    #print(Existing['excess_cap'])
    excess=[]
    for i in Existing['excess_cap']:
        if i >= 0:
            excess.append(0)
        elif i < 0:
            excess.append(i)
    excess = np.asarray(excess)
    #print(len(excess))
    return excess

In [23]:
excess_charging_cap()

array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,  -800.,     0.,     0.,     0., -1200.,
           0.,     0.,  -400.,     0., -2400., -1600.,     0.,  -800.,
           0.,     0., -2800.,     0.,     0.,     0.,     0.,     0.,
           0.,     0., -4000.,     0., -2000.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.])

In [24]:
Existing['new_FCS'] = Existing['existing_num_FCS'].values + to_add
Existing['new_FCS_cap'] = Existing['new_FCS'] * 400
Existing['excess_charge_cap'] = Existing['parking_cap'] - (Existing['scs_cap'] + Existing['new_FCS_cap'])
Existing['excess_charge_cap'].values

array([ 6000.,  6400.,  4400.,  6400.,  6000.,  5200.,  3200.,  8000.,
        7200.,  4000.,   800.,  3200.,  4800.,  6400.,  5600.,  5200.,
        2000.,     0.,  6800.,  5600.,  3600.,  4000.,  4000.,  5200.,
        4000.,  4800.,  5200.,  6400.,  6000.,  7200.,  2800.,  7200.,
        6800.,  6000.,  4400.,   400.,  6400.,  1600.,  1600.,  6000.,
        4000.,  1200.,  6000.,  6400.,     0.,  6800.,  4400.,  1600.,
        4400.,  3600.,  5600.,  -800.,  3200.,  6400.,  7600., -1200.,
        4800.,     0.,  -400.,  3600., -2400., -1600.,  6400.,  -800.,
        2000.,  7200., -2800.,  4800.,  4000.,  4400.,  6800.,  4800.,
        1200.,  5200., -4000.,  6000., -2000.,  2000.,  6800.,  5600.,
         400.,  8400.,  2800.,  3600.,     0.,  4000.,  5600.,  5600.,
        2400.,  7200.,  4000.,  8400.,  4000.,  5200.,  5600.,  6800.,
        6800.,  5600.,  6400.,  4400.])

In [25]:
def adjusted_forecast(Dforecast=Demand_History['2018'].values[:, None], k=k):
    parking_cap = Existing['parking_cap'].values
    parking_cap.shape
    excess_cap = Existing['excess_charge_cap'].values
    excess = excess_charging_cap(Dforecast, k=k)
    newval =  supply_demand().reshape(100, -1)
    #print(newval)
    max_sup_cap = []
    
    for i in range(len(excess)):
        div_factor = excess[i]/(4096)
        #print(div_factor)
        max_sup_cap.append(newval[i] + div_factor)
    max_sup_cap = np.asarray(max_sup_cap)#.reshape(4096, -1)
    
    forecasted = []
    for i in range(4096):
        for cap in max_sup_cap:
            forecasted.append(cap[i])
    forecasted = np.asarray(forecasted)
    return forecasted

In [26]:
refined_forecast = adjusted_forecast()
refined_forecast.sum()

345529.6365968907

In [27]:
forecasty = np.sum(refined_forecast.reshape(4096, -1), 1)
forecasty.shape

(4096,)

In [28]:
infrastructure_to_add = add_infrastructure(Dforecast=forecasty[:, None])
print(infrastructure_to_add.sum())
infrastructure_to_add

175.0


array([3., 0., 0., 2., 0., 0., 0., 0., 4., 1., 6., 1., 2., 1., 0., 1., 5.,
       8., 0., 0., 0., 0., 4., 0., 3., 0., 0., 2., 0., 4., 5., 0., 0., 0.,
       6., 7., 0., 4., 2., 1., 0., 2., 0., 0., 3., 0., 0., 7., 0., 0., 0.,
       6., 2., 0., 0., 7., 0., 6., 2., 3., 5., 8., 0., 1., 5., 0., 6., 0.,
       0., 1., 0., 0., 3., 4., 7., 0., 6., 0., 0., 0., 0., 0., 0., 0., 5.,
       0., 0., 1., 7., 0., 0., 0., 2., 0., 0., 3., 0., 0., 1., 0.])

In [29]:
Demand_History['2018'].values[:, None].sum()

361529.6365968907

# Naive forecasting to test our model and constraint satisfaction
Here I assume that the demand for 2018 is exactly the same
as the demand for 2019 and that there is a +3 increase in all demand values

Forecasting for 2019

The forecast is basically the 2018 data that I used as forecast to evaluate the constraints. All we need to do is to use the result in the refined forecast above and the result in infrastructure_to_add above, for adding infrastructures

In [30]:
infrastructure_to_add = infrastructure_to_add.reshape(-1, 100)
add_fcs_nineteen = infrastructure_to_add # since I optimized for fcs
new_fcs_cap1 = Existing['existing_num_FCS'].values + add_fcs_nineteen
sub_arr = sample_submission
print(sub_arr['value'].values[100:200].sum())
sub_arr['value'].values[100:200] = new_fcs_cap1
print(sub_arr['value'].values[100:200].sum())

619.0
794.0


# For demand values for 2019:

In [31]:
print(sub_arr['value'].values[200:409800].sum())
sub_arr['value'].values[200:409800] = refined_forecast
print(sub_arr['value'].values[200:409800].sum())

361589.5405005043
345529.6365968907


# Forecasting for 2020

In [32]:
twenty_forecast = Demand_History['2018'] + 3
k=10

In [33]:
supply_demand(twenty_forecast.values[:, None])

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [34]:
Existing['scs_cap'] = Existing['existing_num_SCS']*200
Existing['fcs_cap'] = Existing['existing_num_FCS']*400
Existing['total_cap'] = Existing['scs_cap']+ Existing['fcs_cap']
Existing['parking_cap'] = (Existing['total_parking_slots'] - Existing['existing_num_SCS']) * 400 + (Existing['existing_num_SCS']) * 200
Existing['available_cap'] = Existing['parking_cap'] - Existing['total_cap']
add_infrastructure(Dforecast=twenty_forecast.values[:, None])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  3.,  3.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  2.,  0.,  2.,  5.,  0.,  0.,  1.,  3.,  5.,  1.,  1.,  7.,
        2.,  1., 12.,  1.,  1., 11.,  2.,  6., 11.,  0.,  1.,  1.,  9.,
        9.,  0.,  0., 16.,  2., 11., 10.,  4., 11., 13.,  0.,  9., 10.,
        0., 17.,  0.,  2.,  8.,  2.,  4., 10.,  5., 18.,  0.,  9.,  6.,
        0.,  3.,  7.,  0.,  3.,  4.,  1.,  0.,  0.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [35]:
excess_charging_cap(Dforecast=twenty_forecast.values[:, None], k=k)

array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,  -800.,     0.,     0.,     0., -1600.,
           0.,     0.,  -400.,     0., -2400., -2000.,     0.,  -800.,
           0.,     0., -2800.,     0.,     0.,     0.,     0.,     0.,
           0.,     0., -4400.,     0., -2000.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.])

In [36]:
Existing['new_FCS'] = Existing['existing_num_FCS'].values + to_add
Existing['new_FCS_cap'] = Existing['new_FCS'] * 400
Existing['excess_charge_cap'] = Existing['parking_cap'] - (Existing['scs_cap'] + Existing['new_FCS_cap'])
Existing['excess_charge_cap'].values

array([ 6000.,  6400.,  4400.,  6400.,  6000.,  5200.,  3200.,  8000.,
        7200.,  4000.,   800.,  3200.,  4800.,  6400.,  5600.,  5200.,
        2000.,     0.,  6800.,  5600.,  3600.,  4000.,  4000.,  5200.,
        4000.,  4800.,  5200.,  6400.,  6000.,  7200.,  2800.,  7200.,
        6800.,  6000.,  4400.,   400.,  6400.,  1600.,  1600.,  6000.,
        4000.,  1200.,  6000.,  6400.,     0.,  6800.,  4400.,  1600.,
        4400.,  3600.,  5600.,  -800.,  3200.,  6400.,  7600., -1200.,
        4800.,     0.,  -400.,  3600., -2400., -1600.,  6400.,  -800.,
        2000.,  7200., -2800.,  4800.,  4000.,  4400.,  6800.,  4800.,
        1200.,  5200., -4000.,  6000., -2000.,  2000.,  6800.,  5600.,
         400.,  8400.,  2800.,  3600.,     0.,  4000.,  5600.,  5600.,
        2400.,  7200.,  4000.,  8400.,  4000.,  5200.,  5600.,  6800.,
        6800.,  5600.,  6400.,  4400.])

In [37]:
twenty_forecast.values.sum()

373817.6365968907

In [38]:
adjusted_forecast(Dforecast=twenty_forecast.values[:, None], k=k)

array([0.        , 0.        , 5.85623275, ..., 0.        , 0.6789755 ,
       0.        ])

In [39]:
adjusted_forecast()

array([0.        , 0.        , 5.85623275, ..., 0.        , 0.6789755 ,
       0.        ])

In [40]:

refined_forecast_twenty = adjusted_forecast(Dforecast=twenty_forecast.values[:, None], k=k)

In [41]:
forecasty2 = np.sum(refined_forecast_twenty.reshape(4096, -1), 1)
forecasty2.shape

(4096,)

In [42]:
forecasty2

array([ 41.38746297,  98.8781994 , 124.19859965, ...,  54.37862105,
        40.87988231,  92.08054368])

In [43]:
infrastructure_to_add2 = add_infrastructure(Dforecast=forecasty2[:, None])
print(infrastructure_to_add2.sum())
infrastructure_to_add2

173.0


array([3., 0., 0., 2., 0., 0., 0., 0., 4., 1., 6., 1., 2., 1., 0., 1., 5.,
       8., 0., 0., 0., 0., 4., 0., 3., 0., 0., 2., 0., 4., 5., 0., 0., 0.,
       5., 7., 0., 4., 2., 1., 0., 2., 0., 0., 3., 0., 0., 7., 0., 0., 0.,
       6., 1., 0., 0., 7., 0., 6., 2., 3., 5., 8., 0., 1., 5., 0., 6., 0.,
       0., 1., 0., 0., 3., 4., 7., 0., 6., 0., 0., 0., 0., 0., 0., 0., 5.,
       0., 0., 1., 7., 0., 0., 0., 2., 0., 0., 3., 0., 0., 1., 0.])

In [45]:
print(sample_submission['value'].values[409900:410000])

[ 3.  7. 14.  5.  6.  6.  8.  6.  1.  5.  0.  4.  5.  5. 12.  6.  2.  0.
  8.  8.  8.  8.  4.  7.  3. 10. 11.  4.  6.  3.  2.  9. 10.  7.  1.  1.
  9.  4.  4.  6.  6.  4.  4. 10.  6.  6.  8.  1. 10. 13.  8.  1.  7. 16.
 10.  1.  9.  1.  3.  3.  3.  0. 10.  5.  2. 11.  2. 13.  9.  6.  8.  7.
  1.  4.  1. 15.  2. 11.  8.  6.  9.  7.  7.  6.  2.  7.  7.  5.  1. 10.
  5. 12.  3.  6.  9.  4. 10. 14.  5. 11.]


In [46]:
infrastructure_to_add2 = infrastructure_to_add2.reshape(-1, 100)
add_fcs_twenty = infrastructure_to_add2 # since I optimized for fcs
print(Existing['existing_num_FCS'])
new_fcs_cap2 = Existing['existing_num_FCS'].values + add_fcs_twenty
print(new_fcs_cap2)
sub_arr['value'].values[409900:410000] = new_fcs_cap2
print(sub_arr['value'].values[409900:410000])

0      3
1      7
2     14
3      5
4      6
      ..
95     4
96    10
97    14
98     5
99    11
Name: existing_num_FCS, Length: 100, dtype: int64
[[ 6.  7. 14.  7.  6.  6.  8.  6.  5.  6.  6.  5.  7.  6. 12.  7.  7.  8.
   8.  8.  8.  8.  8.  7.  6. 10. 11.  6.  6.  7.  7.  9. 10.  7.  6.  8.
   9.  8.  6.  7.  6.  6.  4. 10.  9.  6.  8.  8. 10. 13.  8.  7.  8. 16.
  10.  8.  9.  7.  5.  6.  8.  8. 10.  6.  7. 11.  8. 13.  9.  7.  8.  7.
   4.  8.  8. 15.  8. 11.  8.  6.  9.  7.  7.  6.  7.  7.  7.  6.  8. 10.
   5. 12.  5.  6.  9.  7. 10. 14.  6. 11.]]
[ 6.  7. 14.  7.  6.  6.  8.  6.  5.  6.  6.  5.  7.  6. 12.  7.  7.  8.
  8.  8.  8.  8.  8.  7.  6. 10. 11.  6.  6.  7.  7.  9. 10.  7.  6.  8.
  9.  8.  6.  7.  6.  6.  4. 10.  9.  6.  8.  8. 10. 13.  8.  7.  8. 16.
 10.  8.  9.  7.  5.  6.  8.  8. 10.  6.  7. 11.  8. 13.  9.  7.  8.  7.
  4.  8.  8. 15.  8. 11.  8.  6.  9.  7.  7.  6.  7.  7.  7.  6.  8. 10.
  5. 12.  5.  6.  9.  7. 10. 14.  6. 11.]


In [47]:
sub_arr['value'].values.sum()

709845.1770973953

In [48]:
sample_submission['value'].values.sum()

709845.1770973953

# For demand values in 2020

In [49]:
print(sub_arr['value'].values[410000:].sum())

361589.5405005043


In [50]:
sub_arr['value'].values[410000:] = refined_forecast
print(sub_arr['value'].values[410000:].sum())

345529.6365968907


In [51]:
Existing.head()

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,scs_cap,fcs_cap,total_cap,parking_cap,available_cap,add_cap,excess_cap,new_FCS,new_FCS_cap,excess_charge_cap
0,0,50.163110,19.412014,23,5,3,1000,1200,2200,8200,6000,0.0,6000.0,3.0,1200.0,6000.0
1,1,37.336451,58.119225,27,4,7,800,2800,3600,10000,6400,0.0,6400.0,7.0,2800.0,6400.0
2,2,46.709232,57.525650,31,6,14,1200,5600,6800,11200,4400,0.0,4400.0,14.0,5600.0,4400.0
3,3,30.528626,55.379835,26,5,5,1000,2000,3000,9400,6400,0.0,6400.0,5.0,2000.0,6400.0
4,4,51.521781,35.116755,32,11,6,2200,2400,4600,10600,6000,0.0,6000.0,6.0,2400.0,6000.0


In [52]:
sub_arr[100:]

,year,data_type,demand_point_index,supply_point_index,value
100,2019,FCS,NaN,0,6.000000
101,2019,FCS,NaN,1,7.000000
102,2019,FCS,NaN,2,14.000000
103,2019,FCS,NaN,3,7.000000
104,2019,FCS,NaN,4,6.000000
...,...,...,...,...,...
819595,2020,DS,4095.0,95,0.000000
819596,2020,DS,4095.0,96,0.000000
819597,2020,DS,4095.0,97,0.000000
819598,2020,DS,4095.0,98,0.678976


In [53]:
Existing['total_slot_19'] = sub_arr['value'][:100] + sub_arr['value'][100:200] 
Existing['total_slot_20'] = sub_arr['value'][409800:409900] + sub_arr['value'][409900:410000] 

In [54]:
#Double checking for constraint 3
Existing.head()
print(Existing.where(Existing['total_parking_slots'] < Existing['total_slot_19']).dropna(axis=0))
print(Existing.where(Existing['total_parking_slots'] < Existing['total_slot_20']).dropna(axis=0))

Empty DataFrame
Columns: [supply_point_index, x_coordinate, y_coordinate, total_parking_slots, existing_num_SCS, existing_num_FCS, scs_cap, fcs_cap, total_cap, parking_cap, available_cap, add_cap, excess_cap, new_FCS, new_FCS_cap, excess_charge_cap, total_slot_19, total_slot_20]
Index: []
Empty DataFrame
Columns: [supply_point_index, x_coordinate, y_coordinate, total_parking_slots, existing_num_SCS, existing_num_FCS, scs_cap, fcs_cap, total_cap, parking_cap, available_cap, add_cap, excess_cap, new_FCS, new_FCS_cap, excess_charge_cap, total_slot_19, total_slot_20]
Index: []


In [55]:
# Double checking to ensure adherence to constraint 3
for i in range(len(sub_arr['value'])):
    if sub_arr['value'][i] < 0:
        sub_arr['value'][i] = 0

C:\Users\OLADEJI\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [56]:
sub_arr.head(15)

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,NaN,0,5.0
1,2019,SCS,NaN,1,4.0
2,2019,SCS,NaN,2,6.0
3,2019,SCS,NaN,3,5.0
4,2019,SCS,NaN,4,11.0
5,2019,SCS,NaN,5,9.0
6,2019,SCS,NaN,6,9.0
7,2019,SCS,NaN,7,6.0
8,2019,SCS,NaN,8,9.0
9,2019,SCS,NaN,9,6.0


In [57]:
sub_arr['value'].head()

0     5.0
1     4.0
2     6.0
3     5.0
4    11.0
Name: value, dtype: float64

In [58]:
sub_arr.to_csv('eighty.csv')

In [59]:
df = pd.read_csv('eighty.csv')
df

,Unnamed: 0,year,data_type,demand_point_index,supply_point_index,value
0,0,2019,SCS,NaN,0,5.000000
1,1,2019,SCS,NaN,1,4.000000
2,2,2019,SCS,NaN,2,6.000000
3,3,2019,SCS,NaN,3,5.000000
4,4,2019,SCS,NaN,4,11.000000
...,...,...,...,...,...,...
819595,819595,2020,DS,4095.0,95,0.000000
819596,819596,2020,DS,4095.0,96,0.000000
819597,819597,2020,DS,4095.0,97,0.000000
819598,819598,2020,DS,4095.0,98,0.678976


In [60]:

for i in range(len(df['value'])):
    if df['value'][i] < 0:
        df['value'][i] = 0

In [61]:
df['value'].min()

0.0

In [62]:
#df.to_csv('seventh1')